In [ ]:
from diff_predictor import data_process, predxgboost, spatial
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 

from os import listdir, getcwd, chdir
from os.path import isfile, join
import os
from sklearn.preprocessing import scale, StandardScaler
from numpy.random import permutation


from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, recall_score, precision_score, f1_score
import operator
import xgboost as xgb
import shap
from xgboost.training import CVPack
from xgboost import callback
from xgboost.core import CallbackEnv
from xgboost.core import EarlyStopException
from xgboost.core import STRING_TYPES

In [ ]:
workbookDir = getcwd()

print('Current Notebook Dir: ' + workbookDir)
chdir(workbookDir) # Go to current workbook Dir"
chdir('..')        # Go up one
chdir('..') 
chdir('..')        # Go up one
print(f'Using current directory for loading data: {getcwd()}')
workbookDir = getcwd()

In [ ]:
#age_feature_path = workbookDir + '/data/raw_data_age/'
region_feature_path = '/Users/nelsschimek/Documents/nancelab/Data/OGD_severity/'
region_feature_filelist = [f for f in listdir(region_feature_path) if isfile(join(region_feature_path, f)) and 'feat' in f and 'cortex' in f]
print(len(region_feature_filelist))

In [ ]:
fstats_tot_age = data_process.generate_fullstats(region_feature_path, region_feature_filelist, ['NT','1_5', '0_5'], 'OGD')

In [ ]:
feature_list = [
    'alpha', # Fitted anomalous diffusion alpha exponenet
    'D_fit', # Fitted anomalous diffusion coefficient
    'kurtosis', # Kurtosis of track
    'asymmetry1', # Asymmetry of trajecory (0 for circular symmetric, 1 for linear)
    'asymmetry2', # Ratio of the smaller to larger principal radius of gyration
    'asymmetry3', # An asymmetric feature that accnts for non-cylindrically symmetric pt distributions
    'AR', # Aspect ratio of long and short side of trajectory's minimum bounding rectangle
    'elongation', # Est. of amount of extension of trajectory from centroid
    'boundedness', # How much a particle with Deff is restricted by a circular confinement of radius r
    'fractal_dim', # Measure of how complicated a self similar figure is
    'trappedness', # Probability that a particle with Deff is trapped in a region
    'efficiency', # Ratio of squared net displacement to the sum of squared step lengths
    'straightness', # Ratio of net displacement to the sum of squared step lengths
    'MSD_ratio', # MSD ratio of the track
    'frames', # Number of frames the track spans
    'Deff1', # Effective diffusion coefficient at 0.33 s
    'Deff2', # Effective diffusion coefficient at 3.3 s
    #'angle_mean', # Mean turning angle which is counterclockwise angle from one frame point to another
    #'angle_mag_mean', # Magnitude of the turning angle mean
    #'angle_var', # Variance of the turning angle
    #'dist_tot', # Total distance of the trajectory
    #'dist_net', # Net distance from first point to last point
    #'progression', # Ratio of the net distance traveled and the total distance
    'Mean alpha', 
    'Mean D_fit', 
    'Mean kurtosis', 
    'Mean asymmetry1', 
    'Mean asymmetry2',
    'Mean asymmetry3', 
    'Mean AR',
    'Mean elongation', 
    'Mean boundedness',
    'Mean fractal_dim', 
    'Mean trappedness', 
    'Mean efficiency',
    'Mean straightness', 
    'Mean MSD_ratio', 
    'Mean Deff1', 
    'Mean Deff2',
    ]

target = 'OGD'

In [ ]:
ecm = fstats_tot_age[feature_list + [target, 'Track_ID', 'X', 'Y']] #dont think i need these rn
print(ecm.shape)
#ecm = ecm[~ecm[list(set(feature_list) - set(['Deff2', 'Mean Deff2']))].isin([np.nan, np.inf, -np.inf]).any(1)]       # Removing nan and inf data points
ecm = ecm[~ecm[list(set(feature_list))].isin([np.nan, np.inf, -np.inf]).any(1)]       # Removing nan and inf data points

ecm.shape

In [ ]:
ecm = ecm.drop_duplicates(subset=['Mean Deff1', 'Mean Deff2'], keep='first') # Remove duplicate track_IDs
ecm.shape

In [ ]:
for feat in feature_list:
    #ecm[feat] = scale(ecm[feat].values)
    print(ecm[feat].mean())

In [ ]:
ecm_filt = ecm[ecm['Mean Deff1'] < 50]
ecm_filt.shape

In [ ]:
def full_preprocess(ecm, balanced=True, y_scramble=False, target=None):

    rand_state = np.random.randint(1, 2000)
    if balanced:
        bal_ecm = data_process.balance_data(ecm, target, random_state=rand_state)
        bal_ecm = bal_ecm.reset_index(drop=True)
        #sampled_df = bal_ecm.sample(frac=0.5)
        sampled_df = data_process.bin_data(bal_ecm)
    else:
        sampled_df = data_process.bin_data(ecm)
    label_df = sampled_df[target]
    features_df = sampled_df.drop([target, 'X', 'Y', 'binx', 'biny', 'bins', 'Track_ID'], axis=1)
    features = features_df.columns

    if y_scramble:
        perm = permutation(len(label_df))
        label_shuffled = label_df[perm]
        le = preprocessing.LabelEncoder()
        sampled_df['encoded_target'] = le.fit_transform(label_shuffled)
    else:
        le = preprocessing.LabelEncoder()
        sampled_df['encoded_target'] = le.fit_transform(sampled_df[target])

    seed = rand_state
    np.random.seed(seed)
    train_split = 0.7
    test_split = 0.5


    training_bins = np.random.choice(sampled_df['bins'].unique(), int(len(sampled_df['bins'].unique())*train_split), replace=False)

    X_train = sampled_df[sampled_df['bins'].isin(training_bins)]
    X_test_val = sampled_df[~sampled_df['bins'].isin(training_bins)]
    X_val, X_test = train_test_split(X_test_val, test_size=test_split, random_state=seed)

    y_train = X_train['encoded_target']
    y_test = X_test['encoded_target']
    y_val = X_val['encoded_target']

    dtrain = xgb.DMatrix(X_train[features], label=y_train)
    dtest = xgb.DMatrix(X_test[features], label=y_test)
    dval = xgb.DMatrix(X_val[features], label=y_val)
    return dtrain, dtest, dval, X_train, X_test, y_train, y_test, le



In [ ]:
param = {'max_depth': 3,
         'eta': 0.005,
         'min_child_weight': 0,
         'verbosity': 0,
         'objective': 'multi:softprob',
         'num_class': 3,
         'silent': 'True',
         'gamma': 5,
         'subsample': 0.15,
         'colsample_bytree': 0.8,
         'eval_metric': "mlogloss",
#          # GPU integration will cut time in ~half:
#          'gpu_id' : 0,
#          'tree_method': 'gpu_hist',
#          'predictor': 'gpu_predictor'
         }

In [ ]:
bal_ecm = data_process.balance_data(ecm, target, random_state=1)
bal_ecm = data_process.bin_data(bal_ecm, resolution=128)
label_df = bal_ecm[target]
features_df = bal_ecm.drop([target, 'Track_ID', 'X', 'Y', 'binx', 'biny', 'bins'], axis=1)
features = features_df.columns

# Regular split

seed = 1234
np.random.seed(seed)
train_split = 0.8
test_split = 0.5

le = preprocessing.LabelEncoder()
bal_ecm['encoded_target'] = le.fit_transform(bal_ecm[target])

training_bins = np.random.choice(bal_ecm.bins.unique(), int(len(bal_ecm.bins.unique())*train_split), replace=False)

X_train = bal_ecm[bal_ecm.bins.isin(training_bins)]
X_test_val = bal_ecm[~bal_ecm.bins.isin(training_bins)]
X_val, X_test = train_test_split(X_test_val, test_size=test_split, random_state=seed)

y_train = X_train['encoded_target']
y_test = X_test['encoded_target']
y_val = X_val['encoded_target']

# dtrain = X_train[features]
# dtest = X_test[features]
# dval = X_val[features]

dtrain = xgb.DMatrix(X_train[features], label=y_train)
dtest = xgb.DMatrix(X_test[features], label=y_test)
dval = xgb.DMatrix(X_val[features], label=y_val)

In [ ]:
best_param = predxgboost.xgb_paramsearch(X_train, y_train, feature_list, init_params=param, num_round=1000, nfold=5, early_stopping_rounds=10, verbose_eval=10)

In [ ]:
print(best_param)

In [ ]:
best_param_cort = {'max_depth': 5, 'eta': 0.01, 'min_child_weight': 10, 'verbosity': 0, 'objective': 'multi:softprob', 'num_class': 3, 'silent': 'True', 'gamma': 0.2, 'subsample': 0.6, 'colsample_bytree': 0.9, 'eval_metric': 'mlogloss'}

In [ ]:
#best_param = {'max_depth': 4, 'eta': 0.005, 'min_child_weight': 0, 'verbosity': 0, 'objective': 'multi:softprob', 'num_class': 3, 'silent': 'True', 'gamma': 5.0, 'subsample': 0.6, 'colsample_bytree': 0.7, 'eval_metric': 'mlogloss'}

In [ ]:
# Currently using parameters found in the diff_mode analysis notebook for age
booster, acc, true_label, preds = predxgboost.train(param, dtrain, dtest, dval, evals=[(dtrain, 'train'), (dval, 'eval')], num_round=1042, verbose=True)


In [ ]:
#class_names = le.classes_
class_names = ['0.5h', '1.5h', 'HC']
class_results = classification_report(true_label, preds, digits=4, target_names = class_names)
print(str(class_results))

In [ ]:
metrics.confusion_matrix(y_test, preds)
plt.figure(figsize=(12,10))
cm_array = metrics.confusion_matrix(true_label, preds)
df_cm = pd.DataFrame(cm_array, index = class_names, columns = class_names)

sns.set(font_scale=1.4) # for label size
ax = sns.heatmap(df_cm, annot=True, annot_kws={"size": 16}, cmap="YlGnBu")
ax.set(xlabel='Predicted', ylabel='Actual')

plt.show()

In [ ]:
explainer = shap.TreeExplainer(booster)
shap_values = explainer.shap_values(X_test[features])

In [ ]:
from matplotlib import colors as plt_colors

explainer = shap.TreeExplainer(booster)
shap_values = explainer.shap_values(X_test[features])
c_NT = '#E69F00'
c_HYase = '#56B4E9'
c_ChABC = '#009E73'

colors = [c_NT, c_HYase, c_ChABC]
class_inds = np.argsort([-np.abs(shap_values[i]).mean() for i in range(len(shap_values))])
cmap = plt_colors.ListedColormap(np.array(colors)[class_inds])
shap.summary_plot(shap_values, X_test[features], class_names=np.array(class_names), max_display=15, title='Total SHAP Values', color=cmap)

In [ ]:
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["font.size"] = 12
plt.rcParams["axes.labelsize"] = 12
plt.rcParams["axes.titlesize"] = 12
plt.rcParams["axes.titleweight"] = "bold"
plt.rcParams["legend.fontsize"] = 12
plt.rcParams["xtick.labelsize"] = 12
plt.rcParams["ytick.labelsize"] = 12

In [ ]:
le.classes_

In [ ]:
shap.summary_plot(shap_values[0], X_test[feature_list], max_display=5, show=False, color_bar=True)
plt.gcf().axes[-1].set_aspect(50)
plt.gcf().axes[-1].set_box_aspect(50)
plt.title(f'Top 5 Features for {class_names[0]}')

In [ ]:
shap.summary_plot(shap_values[1], X_test[feature_list], max_display=5, show=False, color_bar=True)
plt.gcf().axes[-1].set_aspect(50)
plt.gcf().axes[-1].set_box_aspect(50)
plt.title(f'Top 5 Features for {class_names[1]}')
#plt.savefig('striatum_shap_new.pdf', dpi=300, bbox_inches='tight')

In [ ]:
shap.summary_plot(shap_values[2], X_test[feature_list], max_display=5, show=False, color_bar=True)
plt.gcf().axes[-1].set_aspect(50)
plt.gcf().axes[-1].set_box_aspect(50)
plt.title(f'Top 5 Features for {class_names[2]}')

In [ ]:
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"

for i in range(3): 
    figsize = (7.5, 5)
    fig = plt.figure(figsize=figsize)
    # ax = fig.gca()
    print(f'Plotting SHAP values for {le.classes_[i]}')
    shap.summary_plot(shap_values[i], X_test[feature_list], max_display=10, title=f'SHAP Values for {le.classes_[i]}', color=cmap)

In [ ]:
explainer.

In [ ]:
shap.force_plot(explainer.expected_value[0], shap_values[0][0,:], X_test[features].iloc[0,:], matplotlib=True)

In [ ]:
X_display

In [ ]:
for name in X_train[features].columns:
    print(name)
    if 'Mean' in name:
        shap.dependence_plot(ind=name, shap_values=shap_values[0], features=X_test[features])

In [ ]:
shap_interaction_values = explainer.shap_interaction_values(X_test[features])

In [ ]:
shap.dependence_plot(ind='Mean Deff2', shap_values=shap_values[2], features=X_test[features], interaction_index=None)

In [ ]:
shap.dependence_plot(ind='Mean AR', shap_values=shap_values[0], features=X_test[features], interaction_index='auto')

In [ ]:
shap_interaction_values[0].shape

In [ ]:
shap.dependence_plot(("Mean Deff1", "Mean D_fit"), shap_interaction_values[0], X_test[features])#, interaction_index="Mean elongation")

In [ ]:
X_test[features].shape

In [ ]:
shap_interaction_values[0].shape